<a href="https://colab.research.google.com/github/Baby757/NB/blob/main/cifar100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

В данном задании Вам необходимо разработать архитектуру сверточной ИНС, обеспечивающую наибольшую точность при ограничении на количество операций (FLOPs <= 0.707e6). Заготовка кода для выполнения задания приведена выше. Вашей задачей будет заполнить пропущеные места, которые отмечены ключевым словом None. Необходимая точность (accuracy) сети на датасете CIFAR100 - 30% Желаемая точность (accuracy) сети на датасете CIFAR100 - 45%

In [1]:
!pip install keras_flops

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
# Импорт необходимых библиотек
import numpy as np
import pandas as pd
import tensorflow as tf
from keras_flops import get_flops
import matplotlib.pyplot as plt

import tensorflow_addons as tfa
from tensorflow import keras
from tensorflow.keras import layers

In [3]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [4]:
# Глобальные константы
CLASSES       = 100
BATCH_SIZE    = 128
LEARNING_RATE = 1e-2

In [5]:
# Выполните загрузку модели
(X_train, y_train), (X_val, y_val) = tf.keras.datasets.cifar100.load_data()

In [6]:
X_train = X_train.astype('float32')
X_val = X_val.astype('float32')
X_train = X_train / 255.0
X_val = X_val / 255.0

In [7]:
# Преобразуйте метки классов в one_hot формат
y_train = (y_train ==  np.arange(CLASSES)).astype(np.float32)
y_val = (y_val ==  np.arange(CLASSES)).astype(np.float32)

In [8]:
# убедитесь, что данная ячейка выполняется без ошибок
assert X_train.shape == (50000, 32, 32, 3)
assert X_val.shape == (10000, 32, 32, 3)
assert y_train.shape == (50000, 100)
assert y_val.shape == (10000, 100)

In [9]:
print('Shape of X_train is {}'.format(X_train.shape))
print('Shape of X_val is {}'.format(X_val.shape)) 
print('Shape of y_train is {}'.format(y_train.shape))
print('Shape of y_val is {}'.format(y_val.shape))

Shape of X_train is (50000, 32, 32, 3)
Shape of X_val is (10000, 32, 32, 3)
Shape of y_train is (50000, 100)
Shape of y_val is (10000, 100)


In [27]:
# Задайте архитектуру модели
model = tf.keras.models.Sequential([
    tf.keras.Input(shape=[32, 32, 3]),
    tf.keras.layers.Conv2D(32,  kernel_size=(6, 6), strides=(4, 4), padding='same', activation='elu', use_bias=True),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),
    tf.keras.layers.MaxPooling2D(pool_size=(2, 2), strides=2),
    
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(100),
    tf.keras.layers.Activation('softmax'),
    
])


In [28]:
# вычисление количества операций
flops = get_flops(model, batch_size=1)
print(f"FLOPs: {(flops / 1e6):.4f}e6")

FLOPs: 0.5538e6


In [12]:
# вывод краткой информации о модели
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 8, 8, 31)          4588      
                                                                 
 batch_normalization (BatchN  (None, 8, 8, 31)         124       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 8, 8, 31)          0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 4, 4, 31)         0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 496)               0         
                                                                 
 dense (Dense)               (None, 100)               4

In [29]:
# параметры данной ячейки могут быть изменены для получения более высокой точности
model.compile(
    optimizer=tf.keras.optimizers.Adam(
        learning_rate=tf.keras.optimizers.schedules.ExponentialDecay(initial_learning_rate=LEARNING_RATE, decay_steps=1000, decay_rate=0.5)
    ),
    loss=tf.keras.losses.CategoricalCrossentropy(),
    metrics=['accuracy']
)

In [ ]:
model.fit(
    x=X_train,
    y=y_train,
    validation_data=(X_val, y_val),
    batch_size= BATCH_SIZE ,
    callbacks=[
        tf.keras.callbacks.ModelCheckpoint(filepath="{epoch:02d}-{val_accuracy:.2f}.hdf5", save_best_only=True),
        
    ],
    use_multiprocessing=True,
    workers=8,
    epochs=50
) 
# Окончательная оценка модели model1 
scores = model.evaluate(X_val, y_val, verbose=0) 
print("Accuracy: %.2f%%" % (scores[1]*100))

Epoch 1/50
391/391 [==============================] - 9s 24ms/step - loss: 2.4781 - accuracy: 0.3841 - val_loss: 2.8232 - val_accuracy: 0.3175
Epoch 2/50
391/391 [==============================] - 9s 24ms/step - loss: 2.4828 - accuracy: 0.3826 - val_loss: 2.8237 - val_accuracy: 0.3178
Epoch 3/50
391/391 [==============================] - 10s 25ms/step - loss: 2.4814 - accuracy: 0.3846 - val_loss: 2.8223 - val_accuracy: 0.3177
Epoch 4/50
391/391 [==============================] - 11s 27ms/step - loss: 2.4892 - accuracy: 0.3810 - val_loss: 2.8221 - val_accuracy: 0.3183
Epoch 5/50
127/391 [========>.....................] - ETA: 5s - loss: 2.4816 - accuracy: 0.3840

Epoch 41/50 имеет самый хороший результут